In [1]:
def anemia_pipeline(demographics, symptoms, lab_results=None):
    """
    Rule-based anemia diagnostic pipeline.

    demographics: dict with keys
        - age (years)
        - sex ('male', 'female')
        - pregnant (True/False)
        - pregnancy_trimester (1/2/3 or None)

    symptoms: list of symptom strings

    lab_results: dict or None (optional initial run)
        Keys can include: Hb, Hct, RBC, MCV, MCH, MCHC, RDW,
        ferritin, iron, TIBC, platelet_count, B12, folate
    """
    ordered_tests = []
    # ---- STEP 1: Test Ordering ----
    anemia_symptoms = [
        "fatigue", "pallor", "shortness of breath", "dizziness",
        "lightheadedness", "palpitations", "cold extremities",
        "headache", "easy bruising", "bleeding"
    ]
    symptom_count = sum(sym in anemia_symptoms for sym in symptoms)

    if symptom_count >= 2:
        ordered_tests += ["CBC (Hb, Hct, RBC count, MCV, MCH, MCHC, RDW)"]

    if "pallor" in symptoms:
        ordered_tests += ["Serum Ferritin", "Serum Iron", "TIBC"]

    if any(s in symptoms for s in ["easy bruising", "bleeding"]):
        ordered_tests += ["Platelet Count", "Coagulation Profile"]

    if symptom_count >= 3 or any(s in symptoms for s in ["tingling", "numbness"]):
        ordered_tests += ["Vitamin B12", "Folate"]

    if demographics.get("pregnant", False):
        ordered_tests += ["Serum Folate", "Vitamin B12"]

    ordered_tests = list(set(ordered_tests))  # remove duplicates

    # If no lab results provided yet, return the test order
    if lab_results is None:
        return {"Tests to Order": ordered_tests}

    # ---- STEP 2: WHO Hb Cut-offs ----
    def get_hb_cutoff(age, sex, pregnant, trimester):
        if pregnant:
            if trimester == 1: return 11.0
            elif trimester == 2: return 10.5
            elif trimester == 3: return 11.0
        if 0.5 <= age < 5: return 11.0  # 6–59 months
        elif 5 <= age < 12: return 11.5
        elif 12 <= age < 15: return 12.0
        elif sex == "female": return 12.0
        elif sex == "male": return 13.0

    hb_cutoff = get_hb_cutoff(
        demographics["age"],
        demographics["sex"],
        demographics.get("pregnant", False),
        demographics.get("pregnancy_trimester", None)
    )

    hb = lab_results.get("Hb")
    if hb is None:
        return {"Error": "Hb value required for anemia classification."}

    if hb >= hb_cutoff:
        return {
            "Anemia": "No",
            "Hb": hb,
            "Cutoff": hb_cutoff
        }

    # ---- STEP 3: Classification by MCV ----
    mcv = lab_results.get("MCV")
    if mcv is None:
        anemia_type = "Unknown (MCV missing)"
    elif mcv < 80:
        anemia_type = "Microcytic"
    elif 80 <= mcv <= 100:
        anemia_type = "Normocytic"
    else:
        anemia_type = "Macrocytic"

    # ---- STEP 4: Cause Analysis ----
    cause = "Unknown"
    if anemia_type == "Microcytic":
        ferritin = lab_results.get("ferritin")
        iron = lab_results.get("iron")
        tibc = lab_results.get("TIBC")
        if ferritin is not None and ferritin < 30:
            cause = "Iron Deficiency"
        elif iron is not None and tibc is not None and iron < 50 and tibc > 450:
            cause = "Iron Deficiency"
        else:
            cause = "Consider Thalassemia / Chronic Disease"

    elif anemia_type == "Normocytic":
        cause = "Possible acute blood loss / chronic disease"

    elif anemia_type == "Macrocytic":
        b12 = lab_results.get("B12")
        folate = lab_results.get("folate")
        if b12 is not None and b12 < 200:
            cause = "Vitamin B12 Deficiency"
        elif folate is not None and folate < 4:
            cause = "Folate Deficiency"
        else:
            cause = "Consider liver disease / hypothyroidism"

    # ---- STEP 5: Severity Grading ----
    if hb < 8:
        severity = "Severe"
    elif 8 <= hb < 10:
        severity = "Moderate"
    else:
        severity = "Mild"

    # ---- STEP 6: Recommendations ----
    recommendations = []
    if severity == "Severe":
        recommendations.append("Urgent hospital referral, possible transfusion")
    if cause == "Iron Deficiency":
        recommendations.append("Start oral iron therapy and investigate source of loss")
    if cause == "Vitamin B12 Deficiency":
        recommendations.append("Vitamin B12 replacement therapy")
    if cause == "Folate Deficiency":
        recommendations.append("Folate supplementation")

    return {
        "Anemia": "Yes",
        "Hb": hb,
        "Cutoff": hb_cutoff,
        "Type": anemia_type,
        "Cause": cause,
        "Severity": severity,
        "Recommendations": recommendations
    }








In [2]:
# Step 1: Patient comes in with symptoms
demographics = {
    "age": 28,
    "sex": "female",
    "pregnant": False,
    "pregnancy_trimester": None
}
symptoms = ["fatigue", "pallor", "shortness of breath"]

In [3]:
# Step 1 output: Tests to order
print(anemia_pipeline(demographics, symptoms))

{'Tests to Order': ['TIBC', 'Serum Iron', 'CBC (Hb, Hct, RBC count, MCV, MCH, MCHC, RDW)', 'Vitamin B12', 'Folate', 'Serum Ferritin']}


In [4]:
# Step 2: After labs are done
lab_results = {
    "Hb": 9.5,
    "MCV": 72,
    "ferritin": 15,
    "iron": 30,
    "TIBC": 460
}
print(anemia_pipeline(demographics, symptoms, lab_results))


{'Anemia': 'Yes', 'Hb': 9.5, 'Cutoff': 12.0, 'Type': 'Microcytic', 'Cause': 'Iron Deficiency', 'Severity': 'Moderate', 'Recommendations': ['Start oral iron therapy and investigate source of loss']}
